In [ ]:
%%capture --no-stderr
% pip install -U langgraph langchain-community langchain-openai tavily-python pandas

In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")